In [9]:
!python3 -m pip install pip==24.0


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\avril\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
pip install numpy==1.23.5 scikit-learn==1.4.2  joblib pandas

Note: you may need to restart the kernel to use updated packages.


In [10]:
!pip install azureml-sdk

## Carga de Susctiption ID


1. Cargamos el ID de la susctripción de Azure desde las variables de entorno

In [11]:
import os
from dotenv import load_dotenv

load_dotenv()


server_ = os.getenv('AZURE_ID')

2. Creamos un Workspace para hacer el despliegue del modelo dentro del grupo de recursos "resource_group_act3"

In [14]:
from azureml.core import Workspace

ws = Workspace.create(name="workspace_act3_1",
                      subscription_id = server_,
                      resource_group = "resource_group_act3",
                      create_resource_group=False,
                      location = "centralindia")


Deploying KeyVault with name workspackeyvaultf4a4b4bf.
Deploying StorageAccount with name workspacstorage84cf0f0cb.
Deploying AppInsights with name workspacinsights6a617851.
Deployed AppInsights with name workspacinsights6a617851. Took 6.72 seconds.
Deployed KeyVault with name workspackeyvaultf4a4b4bf. Took 23.42 seconds.
Deployed StorageAccount with name workspacstorage84cf0f0cb. Took 29.29 seconds.
Deploying Workspace with name workspace_act3_1.
Deployed Workspace with name workspace_act3_1. Took 40.28 seconds.


3. Cargamos el modelo de RandomForest y el escalador que hace onehotencode para las variables con la librería de scikit learn

In [15]:
from azureml.core.model import Model

mname = "model2"
registered_model = Model.register(model_path="../model/RandomForest_BestModel.pkl",
                                  model_name=mname,
                                  workspace=ws)

# Esto es un pipeline que incluye el modelo entrenado y el preprocesador

Registering model model2


4. Creamos un ambiente virtual de conda con las dependenicas necesarias para correr el modelo y el escalador.

In [16]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

virtual_env = Environment("env-actividad3-1")

virtual_env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['numpy==1.23.5','scikit-learn=1.4.2', 'joblib', 'pandas']
)

5. Cargamos el script que genera el score del modelo, esto junto al ambiente virtual que se creó en el paso anterior.

In [32]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2)


6. Desplegamos el modelo.

In [33]:
service = Model.deploy(workspace=ws,
                       name='service3',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

service.wait_for_deployment()

C:\Users\avril\AppData\Local\Temp\ipykernel_16080\1296210763.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


ACI service creation operation finished, operation "Succeeded"


In [27]:
service.delete()


Running
2025-04-24 01:20:25-06:00 Check and wait for operation (8a955594-f513-492f-a783-6e0d0188e52f) to finish.
2025-04-24 01:20:29-06:00 Deleting service entity.
Succeeded


In [28]:
models = Model.list(workspace=ws)
for model in models:
    print(model.name, model.version)


model2 1


In [37]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_27eda9529773e8b6ef7dc4e79cd37e6f/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_27eda9529773e8b6ef7dc4e79cd37e6f/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_27eda9529773e8b6ef7dc4e79cd37e6f/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2025-04-24T07:25:46,562162300+00:00 - rsyslog/run 
2025-04-24T07:25:46,566722600+00:00 - gunicorn/run 
2025-04-24T07:25:46,569006600+00:00 | gunicorn/run | 
2025-04-24T07:25:46,573944800+00:00 | gunicorn/run | ###############################################
2025-04-24T07:25:46,576081700+00:00 | gunicorn/run | AzureML Container Runtime Information
2025-04-24T07:25:46,582650300+00:00 | gunicorn/run | ###############################################
2025-04-24T07:25:46,584393500+00:00 | gunicorn/run | 
2025-04-24T07:25:46,592057100+00:00 | gunicorn/run | 
bash: /azureml-e

In [35]:
import json

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()